In [1]:
import os
from typing import final
import vg
import numpy as np
from pathlib import Path
from skimage.segmentation import clear_border
from skimage import morphology
from skimage.measure import regionprops_table
from skimage.filters import threshold_otsu
import joblib
from tqdm import tqdm
import pandas as pd
from tifffile import imread, imwrite
from scipy import spatial
import multiprocess as mp
from yaspin import yaspin
import napari

In [2]:
viewer = napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [6]:
def line_from_vect(v, c, length = 50):
    v_points = v * np.mgrid[-length:length:2j][:, np.newaxis]
    v_points += c
    return v_points

def find_vector(pt1,pt2):
    ''' 
    Calculate the vector between two points
    '''
    vect = [0 for c in pt1]
    for dim, coord in enumerate(pt1):
        deltacoord = pt2[dim]-coord
        vect[dim] = deltacoord
    return np.array(vect)

In [4]:
cell_masks = imread('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_filtered_cell_masks.tif') 
viewer.add_labels(cell_masks, name ='cell masks', blending='additive')
emb_cent = np.loadtxt('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_emb_cent.txt')
viewer.add_points(emb_cent, name='embryo centroid', blending = 'additive')
cell_centroid = np.loadtxt('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_cell189_centroid.txt')
viewer.add_points(cell_centroid, name='cell centroid', blending = 'additive')


<Points layer 'cell centroid' at 0x7fb3b1ccba30>

In [7]:
pc0 = np.loadtxt('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_cell189_pc0.txt')
pc1 = np.loadtxt('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_cell189_pc1.txt')
pc2 = np.loadtxt('/Volumes/bigData/wholeMount_volDist/220805-0712_Combined/Cntrl_E01/coords_and_vectors/Cntrl_E01_cell189_pc2.txt')

pc0_points = line_from_vect(pc0, cell_centroid)
pc1_points = line_from_vect(pc1, cell_centroid)
pc2_points = line_from_vect(pc2, cell_centroid)

viewer.add_shapes(pc0_points, shape_type='line', name='pc0', edge_color='green', opacity=0.35, blending='additive')
viewer.add_shapes(pc1_points, shape_type='line', name='pc1', edge_color='red', opacity=0.35, blending='additive')
viewer.add_shapes(pc2_points, shape_type='line', name='pc2', edge_color='blue', opacity=0.35, blending='additive')

<Shapes layer 'pc2' at 0x7fb3b1e76fd0>

In [8]:


v0 = find_vector(emb_cent, cell_centroid)

v0_points = line_from_vect(v0, cell_centroid, length=2)

viewer.add_shapes(v0_points, shape_type='line', name='v0', edge_color='red', opacity=0.35, blending='additive')

<Shapes layer 'v0' at 0x7fb3a07011c0>

In [9]:
print(vg.angle(pc0, v0))
print(180 - vg.angle(pc1, v0))
print(vg.angle(pc2, v0))


47.91885679024104
65.06693724383807
52.34867222318763
